<div align="right" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/Logo blue_dark.png"  style="width:25px" align="right";/>
</div>

# Subqueries and CTEs
© ExploreAI Academy

In this exercise, we will apply subqueries and CTEs in different parts of a query and for different use cases. Ensure that you have downloaded the database file, Northwind.db.

## Learning objectives

By the end of this train, you should:
- Know how to use CTEs to simplify subqueries.
- Understand when to use subqueries and when to use CTEs by comparing their performance and readability.

First, let's load our sample database:

In [1]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql


In [2]:
# Load the Northwind database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///Northwind.db
    

Here is a view of all of our tables in the database:

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Northwind_ERD.png"  style="width:100%";/>
<br>
<br>
    <em>Figure 1: Northwind ERD</em>
</div>

## Exercise

Run the necessary queries that will provide us with the following information. Compare your queries with the solutions at the end of this notebook.

### Exercise 1

Retrieve product details from products that have been ordered by customers from the UK.

In [3]:
%%sql

SELECT
    p.ProductName,
    p.UnitPrice,
    c.ContactName AS CustomerName
FROM
    Products p
    JOIN OrderDetails od ON p.ProductID = od.ProductID
    JOIN orders o ON o.OrderID = od.OrderID
    JOIN customers c ON c.CustomerID = o.CustomerID
WHERE
    c.country = 'UK'
limit 5

 * sqlite:///Northwind.db
Done.


ProductName,UnitPrice,CustomerName
Aniseed Syrup,10,Victoria Ashworth
Wimmers gute Semmelkndel,33.25,Victoria Ashworth
Sasquatch Ale,14,Helen Bennett
Outback Lager,15,Helen Bennett
Jack's New England Clam Chowder,9.65,Helen Bennett


### Exercise 2


Find out the names of customers who have ordered products of more than the average order value.

In [9]:
%%sql

WITH AverageOrderValues AS (
    SELECT
        Customers.CustomerID,
        AVG(OrderDetails.UnitPrice * OrderDetails.Quantity) AS AvgOrderValue
    FROM
        Customers
    JOIN
        Orders ON Customers.CustomerID = Orders.CustomerID
    JOIN
        OrderDetails ON Orders.OrderID = OrderDetails.OrderID
    GROUP BY
        Customers.CustomerID
)

SELECT DISTINCT
    Customers.*
FROM
    Customers
JOIN
    Orders ON Customers.CustomerID = Orders.CustomerID
JOIN
    OrderDetails ON Orders.OrderID = OrderDetails.OrderID
JOIN
    AverageOrderValues ON Customers.CustomerID = AverageOrderValues.CustomerID
WHERE
    OrderDetails.UnitPrice * OrderDetails.Quantity > AverageOrderValues.AvgOrderValue
limit 5


 * sqlite:///Northwind.db
Done.


CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
VINET,Vins et alcools Chevalier,Paul Henriot,Accounting Manager,59 rue de l'Abbaye,Reims,None,51100,France,26.47.15.10,26.47.15.11
TOMSP,Toms Spezialitten,Karin Josephs,Marketing Manager,Luisenstr. 48,Mnster,None,44087,Germany,0251-031259,0251-035695
HANAR,Hanari Carnes,Mario Pontes,Accounting Manager,"Rua do Pao, 67",Rio de Janeiro,RJ,05454-876,Brazil,(21) 555-0091,(21) 555-8765
SUPRD,Suprmes dlices,Pascale Cartrain,Accounting Manager,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium,(071) 23 67 22 20,(071) 23 67 22 21
RICSU,Richter Supermarkt,Michael Holz,Sales Manager,Grenzacherweg 237,Genve,None,1203,Switzerland,0897-034214,None


In [11]:
%%sql 

WITH
    avg_order_value AS (
        SELECT
            AVG(OrderDetails.UnitPrice * OrderDetails.Quantity) AS average_value
        FROM
            OrderDetails
    )
SELECT DISTINCT
    c.*
FROM
    Customers c
    JOIN orders o ON c.CustomerID = o.CustomerID
    JOIN OrderDetails od ON o.OrderID = od.OrderID
    
WHERE
    (od.UnitPrice * od.Quantity) > (SELECT average_value FROM avg_order_value)
limit 5

 * sqlite:///Northwind.db
Done.


CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
TOMSP,Toms Spezialitten,Karin Josephs,Marketing Manager,Luisenstr. 48,Mnster,None,44087,Germany,0251-031259,0251-035695
HANAR,Hanari Carnes,Mario Pontes,Accounting Manager,"Rua do Pao, 67",Rio de Janeiro,RJ,05454-876,Brazil,(21) 555-0091,(21) 555-8765
SUPRD,Suprmes dlices,Pascale Cartrain,Accounting Manager,"Boulevard Tirou, 255",Charleroi,None,B-6000,Belgium,(071) 23 67 22 20,(071) 23 67 22 21
RICSU,Richter Supermarkt,Michael Holz,Sales Manager,Grenzacherweg 237,Genve,None,1203,Switzerland,0897-034214,None
HILAA,HILARION-Abastos,Carlos Hernndez,Sales Representative,Carrera 22 con Ave. Carlos Soublette #8-35,San Cristbal,Tchira,5022,Venezuela,(5) 555-1340,(5) 555-1948


### Exercise 3


Write a CTE to find the most ordered product by each customer.

### Exercise 4

Using a CTE, list employees who have more than the average number of reports.

## Solutions

### Exercise 1

SQL solution with a subquery:

In [10]:
%%sql

SELECT customers.*
FROM customers
WHERE customers.CustomerID IN (
    SELECT orders.CustomerID
    FROM orders
    GROUP BY orders.CustomerID
    ORDER BY COUNT(*) DESC
    LIMIT 1
);

 * sqlite:///Northwind.db
Done.


CustomerID,CompanyName,ContactName,ContactTitle,Address,City,Region,PostalCode,Country,Phone,Fax
SAVEA,Save-a-lot Markets,Jose Pavarotti,Sales Representative,187 Suffolk Ln.,Boise,ID,83720,USA,(208) 555-8097,None


SQL solution with a CTE:

In [ ]:
%%sql

WITH most_orders AS (
    SELECT orders.CustomerID
    FROM orders
    GROUP BY orders.CustomerID
    ORDER BY COUNT(*) DESC
    LIMIT 1
)
SELECT customers.*
FROM customers
JOIN most_orders
ON customers.CustomerID = most_orders.CustomerID;

Both solutions will return the customer with the most orders. The subquery solution nests the logic inside the main query, which can become difficult to read for more complex queries. The CTE solution separates the logic into a different part of the query, which can be more readable, especially for more complex queries.

### Exercise 2

In [11]:
%%sql

WITH avg_order_value AS (
    SELECT AVG(OrderDetails.UnitPrice * OrderDetails.Quantity) AS average_value
    FROM OrderDetails
)
SELECT DISTINCT customers.CompanyName
FROM customers
JOIN orders ON customers.CustomerID = orders.CustomerID
JOIN OrderDetails ON orders.OrderID = OrderDetails.OrderID
WHERE (OrderDetails.UnitPrice * OrderDetails.Quantity) > (SELECT average_value FROM avg_order_value);


 * sqlite:///Northwind.db
Done.


CompanyName
Toms Spezialitten
Hanari Carnes
Suprmes dlices
Richter Supermarkt
HILARION-Abastos
Ernst Handel
Ottilies Kseladen
Blondesddsl pre et fils
Frankenversand
GROSELLA-Restaurante


### Exercise 3

In [13]:
%%sql

WITH most_ordered_products AS (
    SELECT customers.CustomerID, OrderDetails.ProductID, COUNT(*) AS order_count
    FROM customers
    JOIN orders ON customers.CustomerID = orders.CustomerID
    JOIN OrderDetails ON orders.OrderID = OrderDetails.OrderID
    GROUP BY customers.CustomerID, OrderDetails.ProductID
)
SELECT customers.CompanyName, products.ProductName, max_order_count
FROM (
    SELECT CustomerID, MAX(order_count) AS max_order_count
    FROM most_ordered_products
    GROUP BY CustomerID
) AS max_order_count
JOIN most_ordered_products ON max_order_count.CustomerID = most_ordered_products.CustomerID AND max_order_count.max_order_count = most_ordered_products.order_count
JOIN customers ON most_ordered_products.CustomerID = customers.CustomerID
JOIN products ON most_ordered_products.ProductID = products.ProductID
limit 5;


 * sqlite:///Northwind.db
Done.


CompanyName,ProductName,max_order_count
Alfreds Futterkiste,Rssle Sauerkraut,2
Ana Trujillo Emparedados y helados,Queso Cabrales,1
Ana Trujillo Emparedados y helados,Konbu,1
Ana Trujillo Emparedados y helados,Tofu,1
Ana Trujillo Emparedados y helados,Teatime Chocolate Biscuits,1


### Exercise 4

In [13]:
%%sql

WITH avg_reports AS (
    SELECT AVG(report_count) AS average_count
    FROM (
        SELECT COUNT(*) AS report_count
        FROM employees
        JOIN employees AS reports ON employees.EmployeeID = reports.ReportsTo
        GROUP BY employees.EmployeeID
    ) AS report_counts
)
SELECT employees.*
FROM employees
JOIN employees AS reports ON employees.EmployeeID = reports.ReportsTo
GROUP BY employees.EmployeeID
HAVING COUNT(*) > (SELECT average_count FROM avg_reports);

 * sqlite:///Northwind.db
Done.


EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Notes,ReportsTo,PhotoPath,Salary
2,Fuller,Andrew,"Vice President, Sales",Dr.,1952-02-19 00:00:00,1992-08-14 00:00:00,908 W. Capital Way,Tacoma,WA,98401,USA,(206) 555-9482,3457,"Andrew received his BTS commercial in 1974 and a Ph.D. in international marketing from the University of Dallas in 1981. He is fluent in French and Italian and reads German. He joined the company as a sales representative, was promoted to sales manager in January 1992 and to vice president of sales in March 1993. Andrew is a member of the Sales Management Roundtable, the Seattle Chamber of Commerce, and the Pacific Rim Importers Association.",None,http://accweb/emmployees/fuller.bmp,2254.49


<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/ExploreAI_logos/EAI_Blue_Dark.png"  style="width:200px";/>
</div>